# ANÁLISE PREDITIVA DO PREÇO DE VEÍCULOS 

## Descrição:
Primeira atividade do módulo 4 da disciplina 13 - Introdução a Ciência de Dados e Inteligência Artificial, do curso Talento Tech - PR.

## Objetivo:
Implementar um modelo para predição do preço de um veículo com base em suas características, tais como quilometragem, tipo de combustível, ano, etc.

## Ferramentas utilizadas:
O projeto foi desenvolvido em python, com as seguintes bibliotecas:
* Playwright, para raspagem de dados
* Pandas, para manipulação dos dados

---

In [1]:
import pandas as pd

df = 

SyntaxError: invalid syntax (1271191616.py, line 3)